## Step 1. Frame The Problem

In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
!pip install nltk
!pip install konlpy


## Step 2. Import Necessary Datasets

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

import string
import nltk
import re # <- 정규 표현식 re.compile 사용을 위해 라이브러리 사용
from nltk.corpus import stopwords # I, AM, IS, NOT, AN, A 등 수량이나 중요하지 않은 문법적 용어 제거
from nltk.tokenize import word_tokenize # 영어 문장을 단어별로 나눠주기 위해 사용 
from nltk.stem import PorterStemmer # 단어를 원형으로 표현하기 위해, Eating -> eat, ate -> eat 바꿔줌

In [36]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tempe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tempe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [39]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [40]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [41]:
train = train.drop(columns = ['keyword','location'])
train.dtypes

id         int64
text      object
target     int64
dtype: object

In [42]:
train

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


### Analyse the Data

In [43]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7613 non-null   int64 
 1   text    7613 non-null   object
 2   target  7613 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 178.6+ KB


In [45]:
train.isnull().sum()

id        0
text      0
target    0
dtype: int64

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [47]:
test.sample(10)

,id,keyword,location,text
1492,4960,exploded,?????,Fun. Hobby Lobby exploded in this home. http:/...
1582,5348,fire,Lakeland Fl,@traplordthings Y din u fav my fire memez åÀ!
37,122,accident,"Cowtown, Caliii !!",There's a construction guy working on the Disn...
2636,8818,sirens,NaN,I was walking past the fire house today &amp; ...
161,534,army,NaN,One Direction Is my pick for http://t.co/q2eBl...
207,673,attack,NaN,Suspect in latest US theatre attack had psycho...
1752,5917,harm,NaN,Never let the experiences of your past bring h...
2590,8626,seismic,"Calgary, Alberta",Nuclear-Powered Brassiere Explodes During Seis...
1571,5303,fear,"Free Hanseatic City of Bremen, Germany",My grandma had always fear and in some way she...
1925,6489,injuries,NaN,4 Common Running Injuries and How to Avoid The...


In [48]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [49]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [50]:
print(f"Tweets that does not contain information about disaster :\n\n {train.loc[train['target']==0,'text'][0:15].values}", end="\n")

print()

print(f"Tweets that contains information about disaster :\n\n {train.loc[train['target']==1,'text'][0:15].values}", end="\n")

Tweets that does not contain information about disaster :

 ["What's up man?" 'I love fruits' 'Summer is lovely' 'My car is so fast'
 'What a goooooooaaaaaal!!!!!!' 'this is ridiculous....'
 'London is cool ;)' 'Love skiing' 'What a wonderful day!' 'LOOOOOOL'
 "No way...I can't eat that shit" 'Was in NYC last week!'
 'Love my girlfriend' 'Cooool :)' 'Do you like pasta?']

Tweets that contains information about disaster :

 ['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
 'Forest fire near La Ronge Sask. Canada'
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"
 '13,000 people receive #wildfires evacuation orders in California '
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'
 '#flood #disaster Heavy rain causes flash flo

### Working with Columns

In [51]:
def preprocessing(text):
    text = text.lower() # <- 단어를 소문자로
    pattern = re.compile('[^a-z]') # 특수문자&숫자 삭제
    words = nltk.word_tokenize(text) # 
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [PorterStemmer().stem(word) for word in words if word.lower() not in stop_words] 
    preprocessed_text = ' '.join(words)
    return preprocessed_text

train

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [52]:
train['text_cleaned']=train['text'].apply(preprocessing) # aopply를 사용이 가능한 것은 train이 지금 dataframe 객체이기 때문에
test['text']=test['text'].apply(preprocessing)

In [53]:
train

,id,text,target,text_cleaned
0,1,Our Deeds are the Reason of this #earthquake M...,1,deed reason # earthquak may allah forgiv us
1,4,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask . canada
2,5,All residents asked to 'shelter in place' are ...,1,resid ask 'shelter place ' notifi offic . evac...
3,6,"13,000 people receive #wildfires evacuation or...",1,"13,000 peopl receiv # wildfir evacu order cali..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi # alaska smoke # wildfir p...
...,...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridg collaps nearbi home...
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1,@ aria_ahrari @ thetawniest control wild fire ...
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m1.94 [ 01:04 utc ] ? 5km volcano hawaii . htt...
7611,10872,Police investigating after an e-bike collided ...,1,polic investig e-bik collid car littl portug ....
